# Consolidação de Arquivos - Múltipos Layouts e Extensões (OBT)

### Criação do modelo com todos os atributos possíveis

In [58]:
import duckdb
import pandas as pd

In [59]:
df_base_atributos = pd.read_csv(
    "../resultados/detalhes/consolidacao_detalhes.csv",
    sep=";",
    dtype="str"
)

In [61]:
df_base_atributos.head(10)

,caminho,arquivo,coluna,extensao,amostra,quantidade,aba,observacoes,status
0,../arquivos,informacao-cargo.txt,CARGO,.txt,Gerente;Analista,3,NaN,NaN,OK
1,../arquivos,informacao-cargo.txt,ID,.txt,1;2,3,NaN,NaN,OK
2,../arquivos,informacao-cargo.txt,IDADE,.txt,30;25,3,NaN,NaN,OK
3,../arquivos,informacao-cargo.txt,NOME,.txt,Ana;Bruno,3,NaN,NaN,OK
4,../arquivos,informacao-empresas.csv,EMPRESA,.csv,Empresa A;Empresa B,3,NaN,NaN,OK
5,../arquivos,informacao-empresas.csv,ID,.csv,1;2,3,NaN,NaN,OK
6,../arquivos,informacao-empresas.csv,IDADE,.csv,30;25,3,NaN,NaN,OK
7,../arquivos,informacao-empresas.csv,NOME,.csv,Ana;Bruno,3,NaN,NaN,OK
8,../arquivos,informacao-erro.jpg,NaN,.jpg,NaN,NaN,NaN,Erro: Formato de arquivo não suportado,ERRO
9,../arquivos,informacao-projeto.xlsx,ID,.xlsx,1;2,3,Sheet1,NaN,OK


In [62]:
query = """
SELECT
    coluna,
    amostra
FROM (
    SELECT 
            ROW_NUMBER() OVER (PARTITION BY coluna ORDER BY coluna, amostra) AS ID,
            coluna,
            split_part(amostra, ';', 1) amostra
    FROM df_base_atributos
    WHERE coluna IS NOT NULL
    ORDER BY coluna
    ) T1
WHERE ID = 1;
"""

In [63]:
with duckdb.connect(database=":memory:", read_only=False) as conexao:
    df_modelo_colunas = conexao.sql(query).df()

In [64]:
df_modelo_colunas

,coluna,amostra
0,CARGO,Gerente
1,CPF,111.111.111-11
2,DEPARTAMENTO,RH
3,EMAIL,ana@example.com
4,EMPRESA,Empresa A
5,ENDERECO,Rua A 123
6,ESTADO,SP
7,ID,1
8,IDADE,30
9,NOME,Ana


In [65]:
df_modelo_colunas = df_modelo_colunas.pivot(columns="coluna", values="amostra")

In [66]:
#df_modelo_colunas
#df_modelo_colunas.ffill()
df_modelo_colunas.ffill().bfill().iloc[:1]

coluna,CARGO,CPF,DEPARTAMENTO,EMAIL,EMPRESA,ENDERECO,ESTADO,ID,IDADE,NOME,PROJETO,SALARIO,STATUS,TELEFONE
0,Gerente,111.111.111-11,RH,ana@example.com,Empresa A,Rua A 123,SP,1,30,Ana,Projeto SWD,5000,Ativo,123456789


In [67]:
df_modelo_colunas = df_modelo_colunas.ffill().bfill().iloc[:1]

In [68]:
(
    df_modelo_colunas.to_json(
        "../modelo/base/layout_modelo.json",
        orient="records",
        force_ascii=False
    )
)

In [148]:
! mkdir -p ./modelo && \
    datamodel-codegen  \
    --input ../modelo/base/layout_modelo.json \
    --input-file-type json \
    --output ./modelo/modelo.py \
    --class-name TodasColunas \
    --field-constraints

### Conversão de todos os arquivos em JSONs válidos.

In [111]:
import os
from pathlib import Path
from secrets import token_hex
from datetime import datetime

import pandas as pd
import duckdb


def gera_arquivo_json(arquivo: str, aba: str, encoding: str ="latin1") -> bool:
    try:
        if arquivo.endswith(".xlsx") or arquivo.endswith(".xls"):
            df = pd.read_excel(arquivo, engine="openpyxl", sheet_name=aba, dtype="str")
        elif arquivo.endswith(".csv") or arquivo.endswith(".txt"):
            df = pd.read_csv(arquivo, encoding=encoding, delimiter=";", dtype="str")
        elif arquivo.endswith(".json"):
            df = pd.read_json(arquivo)
        else:
            raise ValueError("Formato de arquivo não suportado")
        df["caminho_arquivo"] = str(Path(os.path.relpath(arquivo)).parent)
        df["nome_arquivo"] = arquivo.split("/")[-1]
        df["aba_arquivo"] = aba
        df["datahora_processamento"] = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        caminho_arquivo_json = ("../resultados/tmp/jsons")
        if not os.path.exists(caminho_arquivo_json):
            Path(caminho_arquivo_json).mkdir(parents=True, exist_ok=True)
        arquivo_json = f"{arquivo.split('/')[-1].split('.')[0]}-{token_hex(16)}.json"
        df.to_json(f"{caminho_arquivo_json}/{arquivo_json}", orient="records", force_ascii=False)  
        return True
    except Exception as erro:
        raise ValueError(f"Erro na formatação final do arquivo {arquivo}: {erro}")


In [95]:
detalhes_arquivos = pd.read_csv(
    "../resultados/detalhes/consolidacao_detalhes.csv",
    sep=";",
    dtype="str"
)

In [96]:
detalhes_arquivos = detalhes_arquivos.fillna("")

In [97]:
detalhes_arquivos.head(10)

,caminho,arquivo,coluna,extensao,amostra,quantidade,aba,observacoes,status
0,../arquivos,informacao-cargo.txt,CARGO,.txt,Gerente;Analista,3,,,OK
1,../arquivos,informacao-cargo.txt,ID,.txt,1;2,3,,,OK
2,../arquivos,informacao-cargo.txt,IDADE,.txt,30;25,3,,,OK
3,../arquivos,informacao-cargo.txt,NOME,.txt,Ana;Bruno,3,,,OK
4,../arquivos,informacao-empresas.csv,EMPRESA,.csv,Empresa A;Empresa B,3,,,OK
5,../arquivos,informacao-empresas.csv,ID,.csv,1;2,3,,,OK
6,../arquivos,informacao-empresas.csv,IDADE,.csv,30;25,3,,,OK
7,../arquivos,informacao-empresas.csv,NOME,.csv,Ana;Bruno,3,,,OK
8,../arquivos,informacao-erro.jpg,,.jpg,,,,Erro: Formato de arquivo não suportado,ERRO
9,../arquivos,informacao-projeto.xlsx,ID,.xlsx,1;2,3,Sheet1,,OK


In [98]:
query = """
SELECT DISTINCT caminho, arquivo, aba
FROM detalhes_arquivos
WHERE status = 'OK';
"""

In [99]:
with duckdb.connect(database=":memory:", read_only=False) as conexao:
    arquivos_processamento = conexao.sql(query).fetchall()

In [108]:
arquivos_processamento

[('../arquivos', 'informacao-projeto.xlsx', 'Sheet2'),
 ('../arquivos/dir1', 'informacao-pessoa.json', ''),
 ('../arquivos', 'informacao-empresas.csv', ''),
 ('../arquivos', 'informacao-projeto.xlsx', 'Sheet1'),
 ('../arquivos/dir2/dir2.1', 'informacao-salario.csv', ''),
 ('../arquivos/dir2', 'informacao-endereco.json', ''),
 ('../arquivos/dir1', 'informacao-departamento.xlsx', 'Sheet1'),
 ('../arquivos/dir2', 'informacoes-contatos.txt', ''),
 ('../arquivos', 'informacao-cargo.txt', '')]

In [110]:
for item in arquivos_processamento:
    arquivo = f"{item[0]}/{item[1]}"
    aba = item[2]
    gera_arquivo_json(arquivo, aba)
    print(f"Arquivo: {arquivo} convertido com Sucesso!")

Arquivo: ../arquivos/informacao-projeto.xlsx convertido com Sucesso!
Arquivo: ../arquivos/dir1/informacao-pessoa.json convertido com Sucesso!
Arquivo: ../arquivos/informacao-empresas.csv convertido com Sucesso!
Arquivo: ../arquivos/informacao-projeto.xlsx convertido com Sucesso!
Arquivo: ../arquivos/dir2/dir2.1/informacao-salario.csv convertido com Sucesso!
Arquivo: ../arquivos/dir2/informacao-endereco.json convertido com Sucesso!
Arquivo: ../arquivos/dir1/informacao-departamento.xlsx convertido com Sucesso!
Arquivo: ../arquivos/dir2/informacoes-contatos.txt convertido com Sucesso!
Arquivo: ../arquivos/informacao-cargo.txt convertido com Sucesso!


### Consolidação final dos dados

In [144]:
from modelo.modelo import TodasColunas

In [147]:
teste = TodasColunas()

ValidationError: 14 validation errors for TodasColunas
CARGO
  Field required [type=missing, input_value={}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.8/v/missing
CPF
  Field required [type=missing, input_value={}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.8/v/missing
DEPARTAMENTO
  Field required [type=missing, input_value={}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.8/v/missing
EMAIL
  Field required [type=missing, input_value={}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.8/v/missing
EMPRESA
  Field required [type=missing, input_value={}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.8/v/missing
ENDERECO
  Field required [type=missing, input_value={}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.8/v/missing
ESTADO
  Field required [type=missing, input_value={}

# Referências

+ https://docs.pydantic.dev/latest/integrations/datamodel_code_generator/